In [1]:
import dlib

In [2]:
from skimage import io

In [3]:
from scipy.spatial import distance
import cv2

In [4]:
# загрузим модель
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [5]:
detector = dlib.get_frontal_face_detector()

In [42]:
imtest = io.imread('test1.jpg')
imtest2 = io.imread('test2.jpg')

In [49]:
# Находим
dets = detector(imtest, 1)

In [50]:
win1 = dlib.image_window()
win1.clear_overlay()
win1.set_image(imtest)

In [51]:
for k, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(imtest, d)
    win1.clear_overlay()
    win1.add_overlay(d)
    win1.add_overlay(shape)

Detection 0: Left: 167 Top: 167 Right: 322 Bottom: 322


In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Load VGG Face model weights
model.load_weights('vgg_face_weights.h5')

In [11]:
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output) 

In [12]:
print(imtest.shape)

(460, 460, 3)


In [13]:
dim = (224, 224)
imtest = cv2.resize(imtest, dim, interpolation = cv2.INTER_AREA)

In [14]:
print(imtest.shape)

(224, 224, 3)


In [15]:
imtest = np.expand_dims(imtest, axis=0)

In [16]:
print(imtest.shape)

(1, 224, 224, 3)


In [17]:
vgg_face.predict(imtest)

array([[ 6.3422265,  9.906648 ,  7.9677815, ..., -2.3098629,  2.6143951,
         1.2845223]], dtype=float32)

In [77]:
# дескриптор
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
 
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [78]:
epsilon = 0.40 #cosine similarity
#epsilon = 120 #euclidean distance
 
def verifyFace(img1, img2):
    img1_representation = vgg_face.predict(img1)
    img2_representation = vgg_face.predict(img2)
#     img2_representation = vgg_face.predict(preprocess_image(img2))[0,:]
 
#     cosine_similarity = findCosineDistance(img1_representation, img2_representation)
    euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
#     print(cosine_similarity)
    print(euclidean_distance)
    return(euclidean_distance)
#     if(cosine_similarity &lt; epsilon):
#         print("verified... they are same person")
#     else:
#         print("unverified! they are not same person!")

In [20]:
verifyFace(imtest,imtest)

0.0


In [21]:
imtest = io.imread('test1.jpg')
imtest2 = io.imread('test2.jpg')

In [22]:
print(imtest2.shape)
dim = (224, 224)
imtest2 = cv2.resize(imtest2, dim, interpolation = cv2.INTER_AREA)
print(imtest2.shape)
imtest2 = np.expand_dims(imtest2, axis=0)
print(imtest2.shape)

(1200, 1200, 3)
(224, 224, 3)
(1, 224, 224, 3)


In [23]:
print(imtest.shape)
dim = (224, 224)
imtest = cv2.resize(imtest, dim, interpolation = cv2.INTER_AREA)
print(imtest.shape)
imtest = np.expand_dims(imtest, axis=0)
print(imtest.shape)

(460, 460, 3)
(224, 224, 3)
(1, 224, 224, 3)


In [24]:
verifyFace(imtest,imtest2)

231.58502


In [37]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cv2.imshow("Capture", frame)
cv2.waitKey()
cv2.destroyAllWindows()

In [38]:
cap = cv2.VideoCapture(0)
while (True):
    ret, frame = cap.read()
    cv2.imshow("Capture", frame)
    ch = cv2.waitKey(5)
    if ch == 27:
        break
cv2.destroyAllWindows()

In [62]:
imtest = io.imread('testMy.jpg')

In [73]:
dim = (224, 224)
print(imtest.shape)
imtest = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
imtest = np.expand_dims(imtest, axis=0)
print(imtest.shape)

(224, 224, 3)
(1, 224, 224, 3)


In [82]:
# Находим
color = (255, 0, 0)
while (True):
    ret, frame = cap.read()
    ch = cv2.waitKey(5)
    # Находим
    dets = detector(frame, 1)
    for k, d in enumerate(dets):
#         print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(k, d.left(), d.top(), d.right(), d.bottom()))
        frame = cv2.rectangle(frame, (d.left(), d.top()), (d.right(), d.bottom()), color, 2)
        dim = (224, 224)
        frameForVGG = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        frameForVGG = np.expand_dims(frameForVGG, axis=0)
        cv2.putText(frame, str(verifyFace(imtest,frameForVGG)), (d.left(), d.top()), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)    
    cv2.imshow("Capture", frame)
    ch = cv2.waitKey(5)
    if ch == 27:
        break
cv2.destroyAllWindows()
cap.release()

17.155828
15.37387
16.13153
